<a href="https://colab.research.google.com/github/mmuokubamichael/Mozilla-Luganda-ASR/blob/main/Mozilla_(TEAM_G_O_T)_Fine_Tune_XLS_R_on_Common_Voice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Jan 19 08:09:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive._mount('/content/drive')

Mounted at /content/drive


unzip your train audio file

In [ ]:
from zipfile import ZipFile

file = ZipFile("/content/drive/MyDrive/Train_Audio_mozilla.zip")   
file.extractall("/content") 

creating a csv file that contain path to the audio data 

In [ ]:
import pandas as pd
train = pd.read_csv("/content/drive/MyDrive/uganda_ASR/Train_mozilla.csv")

In [ ]:
train["path"]="/content/validated_dataset/" + train["Clip_ID"] + ".mp3"

In [ ]:
#train.to_csv("/content/drive/MyDrive/uganda_ASR/train_path.csv",index=False)

save to a folder to be used for audion preparation for training our model

In [ ]:
train.to_csv("/content/train_path.csv",index=False)

In [ ]:
%%capture
!pip install datasets==1.13.3
!pip install transformers==4.11.3
!pip install huggingface_hub==0.1
!pip install torchaudio==0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install jiwer

In [ ]:
#from huggingface_hub import notebook_login

#notebook_login()


Then you need to install Git-LFS to upload your model checkpoints:

In [ ]:
#%%capture
#!apt install git-lfs

loading the training and testing data

In [ ]:
from datasets import load_dataset, load_metric, Audio

common_voice_test = load_dataset("common_voice", "lg", split="test")
# saved path to extracted train csv file
sub="/content/train_path.csv"
common_voice_train = load_dataset('csv', data_files=sub)

Downloading:   0%|          | 0.00/4.44k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

...

Using custom data configuration default-b3542b74d667e74e


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-b3542b74d667e74e/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
common_voice_train

DatasetDict({
    train: Dataset({
        features: ['Clip_ID', 'Client_ID', 'up_votes', 'down_votes', 'age', 'gender', 'Target', 'path'],
        num_rows: 13123
    })
})

In [ ]:
common_voice_train = common_voice_train.remove_columns([ "age", "Client_ID", "down_votes","Clip_ID", "gender", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

Let's write a short function to display some random samples of the dataset and run it a couple of times to get a feeling for the transcriptions.

In [ ]:
"""
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))
"""    

'\nfrom datasets import ClassLabel\nimport random\nimport pandas as pd\nfrom IPython.display import display, HTML\n\ndef show_random_elements(dataset, num_examples=10):\n    assert num_examples <= len(dataset), "Can\'t pick more elements than there are in the dataset."\n    picks = []\n    for _ in range(num_examples):\n        pick = random.randint(0, len(dataset)-1)\n        while pick in picks:\n            pick = random.randint(0, len(dataset)-1)\n        picks.append(pick)\n    \n    df = pd.DataFrame(dataset[picks])\n    display(HTML(df.to_html()))\n'

In [ ]:
#show_random_elements(common_voice_train.remove_columns(["path", "audio"]), num_examples=10)

cleaning the *data*

In [ ]:
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\']'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_remove_regex, '', batch["Target"]).lower()
    return batch

In [ ]:
common_voice_train = common_voice_train.map(remove_special_characters)

  0%|          | 0/13123 [00:00<?, ?ex/s]

In [ ]:
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\']'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_remove_regex, '', batch["sentence"]).lower()
    return batch

In [ ]:

common_voice_test = common_voice_test.map(remove_special_characters)

  0%|          | 0/584 [00:00<?, ?ex/s]

Let's look at the processed text labels again.

In [ ]:
#show_random_elements(common_voice_train.remove_columns(["path","audio"]))

loading processor used in train a lauganda from huginface

In [ ]:
from transformers import Wav2Vec2Processor
processor=Wav2Vec2Processor.from_pretrained("lucio/wav2vec2-large-xlsr-luganda")

Downloading:   0%|          | 0.00/158 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/250 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/138 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Next, we can prepare the dataset.

### Preprocess Data

So far, we have not looked at the actual values of the speech signal but just the transcription. In addition to `sentence`, our datasets include two more column names `path` and `audio`. `path` states the absolute path of the audio file. Let's take a look.


XLS-R expects the input in the format of a 1-dimensional array of 16 kHz. This means that the audio file has to be loaded and resampled.

 Thankfully, `datasets` does this automatically by calling the other column `audio`. Let try it out. 

Great, we can see that the audio file has automatically been loaded. This is thanks to the new [`"Audio"` feature](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=audio#datasets.Audio) introduced in `datasets == 4.13.3`, which loads and resamples audio files on-the-fly upon calling.

In the example above we can see that the audio data is loaded with a sampling rate of 48kHz whereas 16kHz are expected by the model. We can set the audio feature to the correct sampling rate by making use of [`cast_column`](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=cast_column#datasets.DatasetDict.cast_column):

In [ ]:
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))

In [ ]:
import torch
import torchaudio

def speech_file_to_array_fn(batch):
  speech_array, sampling_rate = torchaudio.load(batch["path"])
  resampler = torchaudio.transforms.Resample(sampling_rate, 16000)
  batch["speech"] = resampler(speech_array).squeeze().numpy()
  batch["sampling_rate"] = 16000
  return batch
 
common_voice_train = common_voice_train.map(speech_file_to_array_fn)


  0%|          | 0/13123 [00:00<?, ?ex/s]

train prepocessing

In [ ]:
def prepare_dataset(batch):
    #audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [ ]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train['train'].column_names)

  0%|          | 0/13123 [00:00<?, ?ex/s]

In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [ ]:
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names)

  0%|          | 0/584 [00:00<?, ?ex/s]

In [ ]:
#max_input_length_in_sec = 5.0
#common_voice_train = common_voice_train.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

Awesome, now we are ready to start training!

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

Next, the evaluation metric is defined. As mentioned earlier, the 
predominant metric in ASR is the word error rate (WER), hence we will use it in this notebook as well.

In [ ]:
wer_metric = load_metric("wer")

Downloading:   0%|          | 0.00/1.95k [00:00<?, ?B/s]

The model will return a sequence of logit vectors:
$\mathbf{y}_1, \ldots, \mathbf{y}_m$ with $\mathbf{y}_1 = f_{\theta}(x_1, \ldots, x_n)[0]$ and $n >> m$.

A logit vector $\mathbf{y}_1$ contains the log-odds for each word in the vocabulary we defined earlier, thus $\text{len}(\mathbf{y}_i) =$ `config.vocab_size`. We are interested in the most likely prediction of the model and thus take the `argmax(...)` of the logits. Also, we transform the encoded labels back to the original string by replacing `-100` with the `pad_token_id` and decoding the ids while making sure that consecutive tokens are **not** grouped to the same token in CTC style ${}^1$.

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

using trained model by lucio from huginface

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "lucio/wav2vec2-large-xlsr-luganda", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    #vocab_size=len(processor.tokenizer)
)

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

The first component of XLS-R consists of a stack of CNN layers that are used to extract acoustically meaningful - but contextually independent - features from the raw speech signal. This part of the model has already been sufficiently trained during pretraining and as stated in the [paper](https://arxiv.org/pdf/2006.13979.pdf) does not need to be fine-tuned anymore. 
Thus, we can set the `requires_grad` to `False` for all parameters of the *feature extraction* part.

In [ ]:
model.freeze_feature_extractor()

In a final step, we define all parameters related to training. 
To give more explanation on some of the parameters:
- `group_by_length` makes training more efficient by grouping training samples of similar input length into one batch. This can significantly speed up training time by heavily reducing the overall number of useless padding tokens that are passed through the model
- `learning_rate` and `weight_decay` were heuristically tuned until fine-tuning has become stable. Note that those parameters strongly depend on the Common Voice dataset and might be suboptimal for other speech datasets.

For more explanations on other parameters, one can take a look at the [docs](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer#trainingarguments).

During training, a checkpoint will be uploaded asynchronously to the hub every 400 training steps. It allows you to also play around with the demo widget even while your model is still training.

**Note**: If one does not want to upload the model checkpoints to the hub, simply set `push_to_hub=False`.

save to any folder provide the path for the checkpoint

In [ ]:
repo_name = "/content/drive/MyDrive/uganda_ASR_2"

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=11000,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=False,
)

Now, all instances can be passed to Trainer and we are ready to start training!

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train['train'],
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

Using amp fp16 backend




---

${}^1$ To allow models to become independent of the speaker rate, in CTC, consecutive tokens that are identical are simply grouped as a single token. However, the encoded labels should not be grouped when decoding since they don't correspond to the predicted tokens of the model, which is why the `group_tokens=False` parameter has to be passed. If we wouldn't pass this parameter a word like `"hello"` would incorrectly be encoded, and decoded as `"helo"`.

${}^2$ The blank token allows the model to predict a word, such as `"hello"` by forcing it to insert the blank token between the two l's. A CTC-conform prediction of `"hello"` of our model would be `[PAD] [PAD] "h" "e" "e" "l" "l" [PAD] "l" "o" "o" [PAD]`.

### Training

Training will take multiple hours depending on the GPU allocated to this notebook. While the trained model yields somewhat satisfying results on *Common Voice*'s test data of Turkish, it is by no means an optimally fine-tuned model. The purpose of this notebook is just to demonstrate how to fine-tune XLS-R on an ASR dataset.

In case you want to use this google colab to fine-tune your model, you should make sure that your training doesn't stop due to inactivity. A simple hack to prevent this is to paste the following code into the console of this tab (*right mouse click -> inspect -> Console tab and insert code*).

In [ ]:
import IPython.display as ipd
import numpy as np
import random

Depending on what GPU was allocated to your google colab it might be possible that you are seeing an `"out-of-memory"` error here. In this case, it's probably best to reduce `per_device_train_batch_size` to 8 or even less and increase [`gradient_accumulation`](https://huggingface.co/transformers/master/main_classes/trainer.html#trainingarguments).

train took about 22 hours on colab pro

In [ ]:
trainer.train()

In [ ]:
# save the model to any folder to be use for prediction
trainer.save_model("/content/drive/MyDrive/uganda_ASR_2/new_model")

The training loss and validation WER go down nicely.

You can now upload the result of the training to the 🤗 Hub, just execute this instruction:

preparing test data and transcription

the csv test data

In [ ]:
import pandas as pd
test_set=pd.read_csv("/content/drive/MyDrive/uganda_ASR/ASR_test.csv")

In [ ]:
test_set.head()

,Clip_ID,Client_ID,up_votes,down_votes,age,gender,path
0,ID_001WLI2N,byxywujxu77f,2,0,age_group_3,0.0,/content/drive/MyDrive/uganda_ASR/testfile/tes...
1,ID_00M1XUPC,n4bx38dtp45o,2,0,age_group_4,1.0,/content/drive/MyDrive/uganda_ASR/testfile/tes...
2,ID_00O5YT5Y,cs4d5lklb0co,2,0,age_group_4,0.0,/content/drive/MyDrive/uganda_ASR/testfile/tes...
3,ID_00TOYMHF,n4bx38dtp45o,2,0,age_group_4,1.0,/content/drive/MyDrive/uganda_ASR/testfile/tes...
4,ID_0134U5IM,377ac34yi3o0,2,0,NaN,NaN,/content/drive/MyDrive/uganda_ASR/testfile/tes...


extracting path to the test audio the data

In [ ]:
test_set["path"]="/content/drive/MyDrive/uganda_ASR/testfile/test_audio/" + test_set["Clip_ID"] + ".mp3"

In [ ]:
combine_data_train=test_set.drop(["Client_ID","up_votes","down_votes","age","gender"],axis=1)

In [ ]:
combine_data_train.head()

,Clip_ID,path
0,ID_001WLI2N,/content/drive/MyDrive/uganda_ASR/testfile/tes...
1,ID_00M1XUPC,/content/drive/MyDrive/uganda_ASR/testfile/tes...
2,ID_00O5YT5Y,/content/drive/MyDrive/uganda_ASR/testfile/tes...
3,ID_00TOYMHF,/content/drive/MyDrive/uganda_ASR/testfile/tes...
4,ID_0134U5IM,/content/drive/MyDrive/uganda_ASR/testfile/tes...


save the extracted test audio path csv to any folder

In [ ]:
combine_data_train.to_csv("/content/test_path_ASR.csv",index=False)

**transcribing test data**

In [ ]:
%%capture
!pip install datasets==1.13.3
!pip install transformers==4.11.3
!pip install huggingface_hub==0.1
!pip install torchaudio==0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install jiwer

In [ ]:
from datasets import load_dataset, load_metric, Audio

In [ ]:
# test audio path csv file
sub="/content/test_path_ASR.csv"
test_dataset = load_dataset('csv', data_files=sub)

Using custom data configuration default-1f6dbecbbfb3147d


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-1f6dbecbbfb3147d/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from transformers import Wav2Vec2Processor
from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2ForCTC

In [ ]:
#path to save model
model = Wav2Vec2ForCTC.from_pretrained("/content/drive/MyDrive/uganda_ASR_2/new_model").to("cuda")
#processor = Wav2Vec2Processor.from_pretrained("lucio/wav2vec2-large-xlsr-luganda")

loading configuration file /content/drive/MyDrive/uganda_ASR_2/new_model/config.json
Model config Wav2Vec2Config {
  "_name_or_path": "lucio/wav2vec2-large-xlsr-luganda",
  "activation_dropout": 0.05,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForCTC"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 256,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "mean",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": true,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_dropout": 0.0,
  "feat_extract_norm": "layer",
  "feat_proj_dropout": 0.0,
  "feat_quantizer_dropout": 0.0,
  "final_dropout": 

In [ ]:
processor = Wav2Vec2Processor.from_pretrained("lucio/wav2vec2-large-xlsr-luganda")

In [ ]:
import torch
import torchaudio

def speech_file_to_array_fn(batch):
  speech_array, sampling_rate = torchaudio.load(batch["path"])
  resampler = torchaudio.transforms.Resample(sampling_rate, 16_000)
  batch["speech"] = resampler(speech_array).squeeze().numpy()
  batch["sampling_rate"] = sampling_rate
  return batch
 
test_dataset = test_dataset.map(speech_file_to_array_fn)
#common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names)

  0%|          | 0/7067 [00:00<?, ?ex/s]

In [ ]:
processor_pred=[]

In [ ]:

for i in range(0,7067):

  print(i)
  input_dict = processor(test_dataset["train"][i]["speech"],sampling_rate=16_000, return_tensors="pt", padding=True)
  with torch.no_grad():
    logits = model(input_dict.input_values.to("cuda"), attention_mask=input_dict.attention_mask.to("cuda")).logits
  pred_ids = torch.argmax(logits, dim=-1)[0]
  pr_test=processor.decode(pred_ids)
  processor_pred.append(pr_test)
     
"""
  logits = model(input_dict.input_values.to("cuda"), attention_mask=input_dict.attention_mask.to("cuda")).logits
    logits=logits.cpu()
  text = decoder.decode_batch(logits.numpy())
  kenlm_pred.append(text)
"""

0


/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:882: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1


Streaming output truncated to the last 5000 lines.
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2129
2130
2131
2132
2133
2134
2135
2136
2137
2138
2139
2140
2141
2142
2143
2144
2145
2146
2147
2148
2149
2150
2151
2152
2153
2154
2155
2156
2157
2158
2159
2160
2161
2162
2163
2164
2165
2166
2167
2168
2169
2170
2171
2172
2173
2174
2175
2176
2177
2178
2179
2180
2181
2182
2183
2184
2185
2186
2187
2188
2189
2190
2191
2192
2193
2194
2195
2196
2197
2198
2199
2200
2201
2202
2203
2204
2205
2206
2207
2208
2209
2210
2211
2212
2213
2214
2215
2216
2217
2218
2219
2220
2221
2222
2223
2224
2225
2226
2227
2228
2229
2230
2231
2232
2233
2234
2235
2236
2237
2238
2239
2240
2241
2242
2243
2244
2245
2246
2247
2248
2249
2250
2251
2252
2253
2254
2255
2256

'\n  logits = model(input_dict.input_values.to("cuda"), attention_mask=input_dict.attention_mask.to("cuda")).logits\n    logits=logits.cpu()\n  text = decoder.decode_batch(logits.numpy())\n  kenlm_pred.append(text)\n'

In [ ]:
def lm_postprocess(text):
  # my kenlm is borked
  return ' '.join([x if len(x) > 1 else "" for x in text.split()]).strip()

In [ ]:
text_pred = [lm_postprocess(x) for x in processor_pred]

In [ ]:
import pandas as pd
# test audio path csv file
test_set=pd.read_csv("/content/test_path_ASR.csv")

In [ ]:
test_set.head()

,Clip_ID,path
0,ID_001WLI2N,/content/drive/MyDrive/uganda_ASR/testfile/tes...
1,ID_00M1XUPC,/content/drive/MyDrive/uganda_ASR/testfile/tes...
2,ID_00O5YT5Y,/content/drive/MyDrive/uganda_ASR/testfile/tes...
3,ID_00TOYMHF,/content/drive/MyDrive/uganda_ASR/testfile/tes...
4,ID_0134U5IM,/content/drive/MyDrive/uganda_ASR/testfile/tes...


In [ ]:
testing=test_set[["Clip_ID"]]

In [ ]:
testing=test_set[["Clip_ID"]]
testing["Target"]=text_pred

In [ ]:
testing.to_csv("kenlm_wav2vec_last.csv",index=False)